# Day 7: Some Assembly Required

[*Advent of Code 2015 day 7*](https://adventofcode.com/2015/day/7) and [*solution megathread*](https://redd.it/3vr4m4)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/07/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F07%2Fcode.ipynb)

In [53]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

4:1: E402 module level import not at top of file
7:20: E225 missing whitespace around operator


In [54]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [55]:
%load_ext pycodestyle_magic

In [56]:
%pycodestyle_on

## Comments

...

In [57]:
testdata = (['123 -> x',
             '456 -> y',
             'x AND y -> d',
             'x OR y -> e',
             'x LSHIFT 2 -> f',
             'y RSHIFT 2 -> g',
             'NOT x -> h',
             'NOT y -> i'],
            {'d': 72,
             'e': 507,
             'f': 492,
             'g': 114,
             'h': 65412,
             'i': 65079,
             'x': 123,
             'y': 456})

inputdata = downloaded['input'].splitlines()

In [58]:
for data in inputdata:
    print(data)

In [59]:
from enum import Enum, auto


class Gate(Enum):
    INPUTVALUE = 'INPUTVALUE'
    NOT = 'NOT'
    LSHIFT = 'LSHIFT'
    RSHIFT = 'RSHIFT'
    AND = 'AND'
    OR = 'OR'

    def __str__(self):
        return self.value


class CircuitPart:
    outwire = ""
    p1 = p2 = ""
    gate = None

    def __init__(self, circuitstr: str):
        split = circuitstr.split()
        self.outwire = split[-1]
        if len(split) == 3:
            self.gate = Gate.INPUTVALUE
            self.p1 = split[0]
        elif len(split) == 4:
            self.gate = Gate.NOT
            self.p1 = split[1]
        elif len(split) == 5:
            self.gate = Gate[split[1]]
            self.p1 = split[0]
            self.p2 = split[2]

    def __str__(self):
        left = ""
        if self.gate == Gate.INPUTVALUE:
            left = self.p1
        elif self.gate == Gate.NOT:
            left = f'NOT {self.p1}'
        else:
            left = f'{self.p1} {self.gate} {self.p2}'
        return f'{left} -> {self.outwire}'

    def __repr__(self):
        return self.__str__()

In [60]:
for circuitstr in testdata[0]:
    print(CircuitPart(circuitstr))

In [61]:
import numpy as np


wires = {}


def evaluate_part(part: CircuitPart, wire_b_to_a=None) -> bool:
    if part.p1 in wires.keys():
        p1 = wires[part.p1]
    else:
        try:
            p1 = np.uint16(part.p1)
        # The argument was not a known wire but also not an integer
        except ValueError:
            return False

    if part.gate == Gate.INPUTVALUE:
        if wire_b_to_a and part.outwire == 'b':
            wires['b'] = wire_b_to_a
            return True

        wires[part.outwire] = p1
        return True
    elif part.gate == Gate.NOT:
        wires[part.outwire] = np.invert(p1)
        return True

    if part.p2 in wires.keys():
        p2 = wires[part.p2]
    else:
        try:
            p2 = np.uint16(part.p2)
        except ValueError:
            return False

    if part.gate == Gate.LSHIFT:
        wires[part.outwire] = np.left_shift(p1, p2)
        return True
    elif part.gate == Gate.RSHIFT:
        wires[part.outwire] = np.right_shift(p1, p2)
        return True
    elif part.gate == Gate.OR:
        wires[part.outwire] = np.bitwise_or(p1, p2)
        return True
    elif part.gate == Gate.AND:
        wires[part.outwire] = np.bitwise_and(p1, p2)
        return True
    else:
        raise ValueError(f'Could not apply {part} on {wires}')


def evaluate_circuit(circuitparts, wire_b_to_a=None):
    remainingparts = list(circuitparts)[:]
    while len(remainingparts) > 0:
        still_remaining = []
        for part in remainingparts:
            if not evaluate_part(part, wire_b_to_a=wire_b_to_a):
                still_remaining.append(part)
            # else:
            #     print(f'Successfully evaluated {part} for {wires}')

        if len(still_remaining) == len(remainingparts):
            print('Failed to evaluate any remaining circuit parts: ' +
                  f'{still_remaining} for {wires}')
            return

        remainingparts = still_remaining

In [62]:
wires = {}
evaluate_circuit(map(CircuitPart, testdata[0]))
print(wires)

In [63]:
wires = {}
circuitparts = list(map(CircuitPart, inputdata))
evaluate_circuit(circuitparts)
print(wires['a'])

In [64]:
HTML(downloaded['part1_footer'])

In [65]:
HTML(downloaded['part2'])

In [66]:
b = wires['a']
wires = {}
evaluate_circuit(circuitparts, wire_b_to_a=b)
print(wires['a'])

In [67]:
HTML(downloaded['part2_footer'])